<a href="https://colab.research.google.com/github/psygrammer/fast_and_slow/blob/master/drl/notebooks/dqn/tf_ch09_Deep_Q_Network_and_Its_Variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9. Deep Q Network and Its Variants

* Fast And Slow / Deep RL - tf2 [1]
* 김무성

#### 실습 repo

In [4]:
!git clone https://github.com/psygrammer/fast_and_slow

Cloning into 'fast_and_slow'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 59 (delta 13), reused 43 (delta 10), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [5]:
ls

fast_and_slow/  sample_data/


In [13]:
cd /content/fast_and_slow/drl/notebooks/dqn

/content/fast_and_slow/drl/notebooks/dqn


In [14]:
ls

tf_ch09_Deep_Q_Network_and_Its_Variants.ipynb


#### Install dependancies

In [15]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install pyglet==1.3.2 > /dev/null 2>&1
!apt-get install -y xvfb x11-utils python-opengl ffmpeg > /dev/null 2>&1

#### Imports and Helper functions

In [22]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

In [16]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [17]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

-------------------

# Playing Atari games using DQN



In [18]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [19]:
# Let's implement the DQN to play the Ms Pacman game. 
# First, let's import the necessary libraries:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [20]:
# Now, let's create the Ms Pacman game environment using Gym:
env = gym.make("MsPacman-v0")

In [23]:
env = wrap_env(env) # monitoring

In [27]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [24]:
# Set the state size:
state_size = (88, 80, 1)

In [25]:
# Get the number of actions:
action_size = env.action_space.n

#### Preprocess the game screen

Now, let's define a function called preprocess_state which takes the game state (image of the game screen) as an input and returns the preprocessed game state (image of the game screen):

In [28]:
color = np.array([210, 164, 74]).mean()
color

149.33333333333334

In [29]:
def preprocess_state(state):

    #crop and resize the image
    image = state[1:176:2, ::2]

    #convert the image to greyscale
    image = image.mean(axis=2)

    #improve image contrast
    image[image==color] = 0

    #normalize the image
    image = (image - 128) / 128 - 1
    
    #reshape the image
    image = np.expand_dims(image.reshape(88, 80, 1), axis=0)

    return image
